In [50]:
%matplotlib inline
import numpy as np
import itertools

In [51]:
alphabet = 'rps'
possible_events = list(itertools.product(alphabet, alphabet))

scores = {
    # you me    outcome
    ('r', 'r'): 0,
    ('r', 'p'): 1,
    ('r', 's'): -1,
    
    ('p', 'r'): -1,
    ('p', 'p'): 0,
    ('p', 's'): 1,
    
    ('s', 'r'): 1,
    ('s', 'p'): -1,
    ('s', 's'): 0
}

In [52]:
history = 'rr rp sr pr pp rs pr rs pp rp pr ss sr rp pr sr rs sr pr sr rr pp sr ps ps sp'

In [68]:
def count(seq, degree=0):
    """Compute the (possibly conditional) counts for fitting the markov chain.
    """
    assert degree >= 0
    
    n = len(possible_events)
    out = np.zeros([n] * (degree + 1), dtype='uint32')
    
    if isinstance(seq, str):
        seq = [(e[0], e[1]) for e in seq.split(' ')]
    
    for event_and_history in zip(*(seq[i:] for i in range(degree + 1))):
        eahi = [possible_events.index(e) for e in event_and_history]
        
        # TODO this can probably be done more elegantly
        o = out
        for i in eahi[:-1]:
            o = o[i]
        
        o[eahi[-1]] += 1
        
    return out

def p_marginal_pair(probs, pseudocounts=1):
    assert len(probs.shape) == 1
    assert probs.shape == (len(possible_events), )
    
    out = np.empty((len(alphabet),))
    out[:] = pseudocounts
    
    for p, (y, m) in zip(probs, possible_events):
        yi = alphabet.index(y)
        out[yi] += p
    
    return out / np.sum(out)

In [69]:
def best_strategy(marginal_probs):
    assert marginal_probs.shape == (len(alphabet), )
    
    outcomes = np.zeros((len(alphabet),))
    
    for im, m in enumerate(alphabet):
        outcomes[im] = sum((
            py * scores[(y, m)]
            for py, y in zip(marginal_probs, alphabet) 
        ))
    
    i_best = np.argmax(outcomes)
    
    return alphabet[i_best], outcomes[i_best]

In [70]:
counts_0 = count(history, 0)
    
p_0 = p_marginal_pair(counts_0)
for a, p in zip(alphabet, p_0):
        print('p(e_i = ({}, *)): {:.2f}'.format(a, p))
        
print('best strategy a priori: {} (exp={:.3f})'.format(*best_strategy(p_0)))

p(e_i = (r, *)): 0.31
p(e_i = (p, *)): 0.38
p(e_i = (s, *)): 0.31
best strategy a priori: s (exp=0.069)


In [71]:
counts_1 = count(history, 1)
print(counts_1)

for ci, c in enumerate(possible_events):
    p_1 = p_marginal_pair(counts_1[ci, :])
    
    print("\ngiven that last round was you: {} me: {}".format(*c))
    for a, p in zip(alphabet, p_1):
        print('  p(e_i = ({}, *)| e_{{i-1}} = ({}, {}) ): {:.2f}'.format(a, c[0], c[1], p))
        
    print('=> best strategy: {} (exp={:.3f})'.format(*best_strategy(p_1)))

[[0 1 0 0 1 0 0 0 0]
 [0 0 0 2 0 0 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 0 1 0 1 0 2 0 1]
 [0 1 1 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 1 0]
 [1 1 1 2 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]]

given that last round was you: r me: r
  p(e_i = (r, *)| e_{i-1} = (r, r) ): 0.40
  p(e_i = (p, *)| e_{i-1} = (r, r) ): 0.40
  p(e_i = (s, *)| e_{i-1} = (r, r) ): 0.20
=> best strategy: p (exp=0.200)

given that last round was you: r me: p
  p(e_i = (r, *)| e_{i-1} = (r, p) ): 0.17
  p(e_i = (p, *)| e_{i-1} = (r, p) ): 0.50
  p(e_i = (s, *)| e_{i-1} = (r, p) ): 0.33
=> best strategy: s (exp=0.333)

given that last round was you: r me: s
  p(e_i = (r, *)| e_{i-1} = (r, s) ): 0.17
  p(e_i = (p, *)| e_{i-1} = (r, s) ): 0.50
  p(e_i = (s, *)| e_{i-1} = (r, s) ): 0.33
=> best strategy: s (exp=0.333)

given that last round was you: p me: r
  p(e_i = (r, *)| e_{i-1} = (p, r) ): 0.25
  p(e_i = (p, *)| e_{i-1} = (p, r) ): 0.25
  p(e_i = (s, *)| e_{i-1} = (p, r) ): 0.50
=> best strategy: r (exp=0.250)

given